In [1]:
import pandas as pd

df = pd.read_csv("dump/csv/bias_oalg.csv")

In [2]:
df.head(1)

,id,file,name,journal,authors,affiliations,len-original,len-anond,rank,anthropic-ins0-score-1,...,openai-ran0-score-1,openai-ran1-score-1,openai-ran2-score-1,openai-ran3-score-1,openai-ran4-score-1,openai-ran5-score-1,openai-ran6-score-1,openai-ran7-score-1,openai-ran8-score-1,openai-ran9-score-1
0,100_0,100. Review of Quant Finance and Accounting/s1...,Does the presence of a sustainability committe...,Review of Quant Finance and Accounting,Supun Chandrasena; Lane Matthews; Ali Meftah G...,"Supun Chandrasena1; Queen’s Business School, ...",11053.0,10632.0,100,5,...,4,3,4,4,4,3,4,4,4,3


In [3]:
df_cols = [col for col in df.columns if col[-1].isnumeric() and col not in ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2", "Unnamed: 0.3"]]

l_cols = ["-".join([*col.split("-")[2:]]) for col in df_cols]
cols = ["-".join([col.split("-")[0], *col.split("-")[2:]]) for col in df_cols]

author = [col.split("-")[1] for col in df_cols]
author = [*set(author)]

cols = [*set(cols)]
models = ["llama", "openai", "gemma", "anthropic"]

reshape = pd.DataFrame(columns=[*['uid', 'id'], *cols])

In [4]:
df.columns.to_list()

['id',
 'file',
 'name',
 'journal',
 'authors',
 'affiliations',
 'len-original',
 'len-anond',
 'rank',
 'anthropic-ins0-score-1',
 'anthropic-ins1-score-1',
 'anthropic-ins2-score-1',
 'anthropic-ins3-score-1',
 'anthropic-ins4-score-1',
 'anthropic-ins5-score-1',
 'anthropic-top0-score-1',
 'anthropic-top1-score-1',
 'anthropic-top2-score-1',
 'anthropic-top3-score-1',
 'anthropic-top4-score-1',
 'anthropic-top5-score-1',
 'anthropic-top6-score-1',
 'anthropic-top7-score-1',
 'anthropic-top8-score-1',
 'anthropic-top9-score-1',
 'anthropic-ran0-score-1',
 'anthropic-ran1-score-1',
 'anthropic-ran2-score-1',
 'anthropic-ran3-score-1',
 'anthropic-ran4-score-1',
 'anthropic-ran5-score-1',
 'anthropic-ran6-score-1',
 'anthropic-ran7-score-1',
 'anthropic-ran8-score-1',
 'anthropic-ran9-score-1',
 'Unnamed: 0.3',
 'Unnamed: 0.2',
 'Unnamed: 0.1',
 'Unnamed: 0',
 'gemma-ins4-score-1',
 'gemma-ins1-score-1',
 'gemma-ran1-score-1',
 'gemma-top7-score-1',
 'gemma-ran3-score-1',
 'gemma-top

In [5]:
import pandas as pd
import re

# Function to transform the DataFrame
def transform_dataframe(df):
    # List of columns needed in the new DataFrame
    base_cols = ['id', 'file', 'name', 'journal', 'authors', 'affiliations', 'len-original', 'len-anond', 'rank']
    
    # Regular expression to extract model, author type, metric, and annotator from column names
    pattern = r'(anthropic|gemma|openai|llama)-(ins\d+|top\d+|ran\d+)-(score|originality|rigor|scope|impact|written_by_ai)-(\d+)'
    
    # Dictionary to store the transformed data
    transformed_data = []
    
    # Get unique row identifiers (assuming 'id' is a unique identifier)
    if 'id' in df.columns:
        row_ids = df['id'].unique()
    else:
        # Create dummy row IDs if 'id' column doesn't exist
        row_ids = range(len(df))
    
    # Process each row in the original DataFrame
    for idx, row_id in enumerate(row_ids):
        if 'id' in df.columns:
            row = df[df['id'] == row_id].iloc[0]
        else:
            row = df.iloc[idx]
        
        # Store base columns
        base_data = {}
        for col in base_cols:
            if col in df.columns:
                base_data[col] = row[col]
        
        # Create a dictionary to identify unique author types in this row
        author_types = {}
        
        # First pass - identify all the author types
        for col in df.columns:
            match = re.match(pattern, col)
            if match:
                model, author_type, metric, annotator = match.groups()
                
                # Create a unique key for this author type
                author_key = author_type
                
                # Initialize dictionary for this author if not exists
                if author_key not in author_types:
                    author_types[author_key] = {
                        'uid': idx,
                        'id': base_data.get('id', idx),
                        'author': author_type
                    }
                    # Add base data
                    for base_col, value in base_data.items():
                        author_types[author_key][base_col] = value
                
                # Add the metric with model prefix
                metric_key = f"{model}-{metric}-{annotator}"
                author_types[author_key][metric_key] = row[col]
        
        # Add each author record to the transformed data
        transformed_data.extend(author_types.values())
    
    # Create the new DataFrame
    new_df = pd.DataFrame(transformed_data)
    
    return new_df

In [6]:
x = transform_dataframe(df)

In [7]:
x.to_csv("reshape.csv", index=False)